In [2]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
import os
import openai


import warnings
warnings.filterwarnings("ignore")

from langchain.agents import tool
from datetime import date

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.environ['OPENAI_API_KEY']
openai.api_key= os.environ['OPENAI_API_KEY']

print('lib and key are loaded')

lib and key are loaded


In [ ]:

# # tools = load_tools(["weather-api"], llm=llm, listen_api_key="...")
# agent2 = initialize_agent(tools2,
#                          llm,
#                          agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#                          verbose=True)

### "`chain.run` supports only one positional argument."
* without tool, llm cannot provide real-time info

In [7]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

# Chain2 - report weather
template = """ What's the weather today for {country}?
Country: 
{country}
Current weather:"""
prompt_template = PromptTemplate(input_variables=["country"], template=template)
chain_two = LLMChain(llm=llm, prompt=prompt_template) 

chain_two.run("What's the weather forecast for Singapore today?")

# Chain1 - solve date 
# chain_one = agent
# from langchain.chains import SimpleSequentialChain
# overall_chain = SimpleSequentialChain(
#                   chains=[chain_one, chain_two],
#                   verbose=True)

'I\'m sorry, I cannot provide real-time information as I am an AI language model and do not have access to current weather data. However, you can easily check the weather forecast for Singapore today by using a weather website or app, or by searching "weather forecast for Singapore today" on a search engine.'

### agent with multiple tools, one of which is customized

In [15]:
import pandas as pd
str(date.today() + pd.DateOffset(days=2))[:10]

'2023-07-16'

In [20]:
'a'.values

AttributeError: 'str' object has no attribute 'values'

In [18]:
def get_date(param) -> str:
    """ get the date of a future date {days} from today """
    import pandas as pd
    return str(date.today() + pd.DateOffset(days=2))[:10]

dateTool = Tool(
    name="get_date",
    func=get_date,
    description="use to get a future date"
)

llm = OpenAI(temperature=0)
search = SerpAPIWrapper()

# Define a list of tools offered by the agent
serp_tool = Tool(
        name="Search",
        func=search.run,
        description="Useful when you need to answer questions about current events. You should ask targeted questions.")

tools = [dateTool, serp_tool]

agent= initialize_agent(
    tools,
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [19]:
import langchain
langchain.debug=True
agent.run("What's the wather in Singapore in 3 days")


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What's the wather in Singapore in 3 days"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What's the wather in Singapore in 3 days",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nget_date: use to get a future date\nSearch: Useful when you need to answer questions about current events. You should ask targeted questions.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the \"action\" field are: get_date, Search\n\nThe $JSO

AttributeError: 'int' object has no attribute 'values'

In [8]:
print(agent.agent.llm_chain.prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nget_date: use to get a future date\nSearch: Useful when you need to answer questions about current events. You should ask targeted questions.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: get_date, Search\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nALWAYS use the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction:\

### chaining

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

# Chain2 - report weather
template = """ What's the weather today for {country} on {date}.
Date:
{date}
Country: 
{country}
Current weather:"""
prompt_template = PromptTemplate(input_variables=["country", "date"], template=template)
chain_two = LLMChain(llm=llm, prompt=prompt_template) 

chain_two.run("What's the weather forecast for Singapore today?")

# Chain1 - solve date 
# chain_one = agent
# from langchain.chains import SimpleSequentialChain
# overall_chain = SimpleSequentialChain(
#                   chains=[chain_one, chain_two],
#                   verbose=True)

ValueError: A single string input was passed in, but this chain expects multiple inputs ({'country', 'date'}). When a chain expects multiple inputs, please call it by passing in a dictionary, eg `chain({'foo': 1, 'bar': 2})`

### hit api simple, without an agent

In [8]:
from langchain.chains.api import open_meteo_docs
from langchain.chains import APIChain


chain_new = APIChain.from_llm_and_api_docs(llm, "https://cat-fact.herokuapp.com/facts/", verbose=True)
chain_new.run("Does wikipedia has cat sound recording?")



> Entering new  chain...
https://cat-fact.herokuapp.com/facts/?has_sound=true
[{"status":{"verified":true,"sentCount":1},"_id":"58e008780aac31001185ed05","user":"58e007480aac31001185ecef","text":"Owning a cat can reduce the risk of stroke and heart attack by a third.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-29T20:20:03.844Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"58e009390aac31001185ed10","user":"58e007480aac31001185ecef","text":"Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat the standard: clean, cool drinking water.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-04T21:20:02.979Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"588e746706ac2b00110e59ff","user":"588e6e8806ac2b00110e59c3","text":"Domestic cats spend 

' The response from the API indicates that cats do have sound recordings, as the API call returned multiple facts about cats that include the fact that cats have a purr frequency that is the same as the frequency at which muscles and bones repair themselves.'